In [ ]:
# Add Econ-ARK logo header
from IPython.display import HTML, display

logo_html = """
<style>
  .ark-header {display:flex;align-items:center;
               background:#005b8f;padding:20px 24px;
               position:sticky;top:0;z-index:1000;
               box-shadow:0 2px 8px rgba(0,0,0,0.15);}
  .ark-header img {height:48px;margin-right:20px;}
  .ark-header span {color:#fff;font:600 1.35rem/1 system-ui,sans-serif;letter-spacing:-0.01em;}
</style>
<div class='ark-header'>
  <a href='https://econ-ark.org' target='_blank' style='border:0'>
    <img src='https://econ-ark.org/assets/img/econ-ark-logo-white.png'
         alt='Econ‑ARK logo'>
  </a>
  <span>HANK‑SAM Interactive Dashboard</span>
</div>
"""
display(HTML(logo_html))


In [ ]:
# HANK-SAM Model Interactive Dashboard.
# Author: Alan Lujan <alujan@jhu.edu>

# This Voila dashboard allows interactive exploration of the HANK-SAM model's
# fiscal multipliers under different monetary and fiscal policy parameters.

# Import required packages
import ipywidgets as widgets
from IPython.display import clear_output, display
from ipywidgets import HTML, HBox, Layout, VBox
import matplotlib.pyplot as plt
from cycler import cycler

# Import our refactored model module
import hank_sam as hs

# Define Econ-ARK brand colors and styling
ARK_BLUE = "#005b8f"      # primary
ARK_LIGHTBLUE = "#0ea5e9" # lighter accent
ARK_SLATE = "#1e293b"     # dark text
ARK_ORANGE = "#f97316"    # accent
ARK_GREY = "#6b7280"      # utility
ARK_LIGHT = "#f8fafc"     # background
# CSS can use system fonts, but matplotlib needs actual installed fonts
CSS_FONT_STACK = "system-ui,-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Ubuntu,'Helvetica Neue',Arial,sans-serif"
MPL_FONT_STACK = ["DejaVu Sans", "Arial", "Helvetica", "sans-serif"]

# Configure Matplotlib with Econ-ARK branding
plt.rcParams.update({
    "font.family": MPL_FONT_STACK,
    "axes.titlesize": 12,
    "axes.titleweight": "bold",
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "axes.prop_cycle": cycler(color=[ARK_BLUE, ARK_ORANGE, ARK_LIGHTBLUE, ARK_SLATE, "#0d9488"])
})

def tidy_legend(fig):
    """Helper to format legends consistently across all figures."""
    handles, labels = fig.axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, 
              bbox_to_anchor=(0.5, -0.12),
              loc='center',
              ncol=3,
              prop={'size': 9})
    fig.subplots_adjust(bottom=0.2)

# Global style for headings and typography
display(HTML("""
<style>
    :root {
        /* Brand colors */
        --ark-blue: #005b8f;          /* primary */
        --ark-lightblue: #0ea5e9;     /* lighter accent */
        --ark-slate: #1e293b;         /* dark text */
        --ark-body: 0.95rem;          /* base text size */
    }

    /* ===== HEADING SCALE & DECORATION ===== */
    h1, .ark-h1 {
        font:700 1.65rem/1.3 system-ui,sans-serif;
        color:var(--ark-blue);
    }

    h2, .ark-h2 {
        font:600 1.35rem/1.35 system-ui,sans-serif;
        color:var(--ark-slate);
        position:relative;
        margin-bottom:1.1rem;
    }
    h2::after, .ark-h2::after {              /* thin underline bar */
        content:'';
        position:absolute;
        left:0; bottom:-6px;
        width:48px; height:3px;
        background:var(--ark-blue);
        border-radius:2px;
    }

    /* Optional lighter accent for figures */
    .ark-h2.lightblue { color:var(--ark-lightblue); }
    .ark-h2.lightblue::after { background:var(--ark-lightblue); }

    h3, .ark-h3 {
        font:600 1.15rem/1.4 system-ui,sans-serif;
        color:var(--ark-slate);
    }

    /* Body and utility classes */
    p.ark-body {font-size:var(--ark-body);line-height:1.45;color:var(--ark-slate);}
    .ark-num {color:var(--ark-blue);font-weight:600;}
    .ark-label {font-size:0.9rem;font-weight:500;color:#334155;}
</style>
"""))

# Helper function to create heading widgets
def create_heading(text, level=2, style_class=""):
    """Create a heading widget with consistent styling."""
    tag = f"h{level}"
    class_str = f"ark-h{level} {style_class}".strip()
    return HTML(f"<{tag} class='{class_str}'>{text}</{tag}>")

In [ ]:
# Create style for sliders - optimized for compact layout
style = {
    "description_width": "45%",  # Relative description width
    "description_font_size": "0.9rem",  # Match ark-label class
    "description_font_weight": "500",  # Match ark-label class
    "description_color": "#334155",  # slate-700 per brand guide
}
slider_layout = Layout(width="90%")  # Wider relative width per brand guide

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# SECTION 1: CREATE PARAMETER WIDGETS
# ═════════════════════════════════════════════════════════════════════════════

# Monetary Policy Parameters
phi_pi_widget = widgets.FloatSlider(
    value=1.5,
    min=1.0,
    max=3.0,
    step=0.1,
    description="Taylor Rule π  (φπ):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".2f",
)

In [ ]:
phi_y_widget = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.05,
    description="Taylor Rule Y (φy):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".2f",
)

In [1]:
# ═════════════════════════════════════════════════════════════════════════════
# SECTION 1: CREATE PARAMETER WIDGETS
# ═════════════════════════════════════════════════════════════════════════════

# Monetary Policy Parameters
phi_pi_widget = widgets.FloatSlider(
    value=1.5,
    min=1.0,
    max=3.0,
    step=0.1,
    description="Taylor Rule π (φπ):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".2f",
)

NameError: name 'widgets' is not defined

In [2]:
phi_y_widget = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.05,
    description="Taylor Rule Y (φy):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".2f",
)

NameError: name 'widgets' is not defined

In [ ]:
rho_r_widget = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.95,
    step=0.05,
    description="Taylor Rule inertia (ρr):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".2f",
)

kappa_p_widget = widgets.FloatSlider(
    value=0.06191950464396284,
    min=0.01,
    max=0.2,
    step=0.005,
    description="Phillips curve slope (κp):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".3f",
)

# Fiscal and Structural Parameters
phi_b_widget = widgets.FloatSlider(
    value=0.015,
    min=0.0,
    max=0.1,
    step=0.005,
    description="Fiscal adjustment (φb):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".3f",
)

real_wage_rigidity_widget = widgets.FloatSlider(
    value=0.837,
    min=0.0,
    max=1.0,
    step=0.05,
    description="Real wage rigidity:",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
    readout_format=".3f",
)

# Policy Duration Parameters
ui_extension_widget = widgets.IntSlider(
    value=4,
    min=1,
    max=12,
    step=1,
    description="UI extension (quarters):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
)

tax_cut_widget = widgets.IntSlider(
    value=8,
    min=1,
    max=16,
    step=1,
    description="Tax cut (quarters):",
    style=style,
    layout=slider_layout,
    continuous_update=False,
    readout=True,
)

In [ ]:
# Create a status panel with button and progress label
status_panel = widgets.VBox([
    widgets.Button(
        description="▶ Run Simulation",
        layout=Layout(width="90%", height="2.5em"),
        style={"button_color": ARK_LIGHTBLUE, "font_weight": "600"},
    ),
    widgets.Label(
        value="Ready to run simulation",
        layout=Layout(width="90%", padding="0.5em 0"),
        style={
            "color": ARK_SLATE,
            "font_size": "0.9rem",
            "font_weight": "500",
            "text_align": "center"
        }
    )
])

# Extract individual widgets for use in callbacks
run_button = status_panel.children[0]
progress_label = status_panel.children[1]

In [ ]:
# Create placeholder message for plots
placeholder_html = f"""
<div style="display:flex; flex-direction:column; align-items:center; justify-content:center; 
            background-color:{ARK_LIGHT}; border-radius:8px; padding:1.5em;
            height:250px;">
    <div style="font-size:1.1rem; color:{ARK_SLATE}; margin-bottom:0.75em;">
        ⚡ Run Simulation to Generate Plots
    </div>
    <div style="font-size:0.9rem; color:{ARK_GREY}; text-align:center; max-width:300px;">
        Adjust parameters and click 'Run Simulation' to explore fiscal policy scenarios.
    </div>
</div>
"""

# Output widgets for truly responsive figures - adaptive to container size
fig1_output = widgets.Output(
    layout=Layout(
        width="100%",
        height="320px",  # More height for plot
        min_height="320px",  # Ensure minimum height
        background_color=ARK_LIGHT,  # Light background for plots
        border_radius="8px",  # Rounded corners
        padding="1.5em 1em",  # More top/bottom padding
        margin="0.5em 0",  # More vertical spacing
        overflow="visible",  # Allow content to overflow
        display="flex",  # Use flexbox
        align_items="center",  # Center vertically
        justify_content="center",  # Center horizontally
    )
)
fig2_output = widgets.Output(
    layout=Layout(
        width="100%",
        height="320px",  # More height for plot
        min_height="320px",  # Ensure minimum height
        background_color=ARK_LIGHT,  # Light background for plots
        border_radius="8px",  # Rounded corners
        padding="1.5em 1em",  # More top/bottom padding
        margin="0.5em 0",  # More vertical spacing
        overflow="visible",  # Allow content to overflow
        display="flex",  # Use flexbox
        align_items="center",  # Center vertically
        justify_content="center",  # Center horizontally
    )
)

# Initialize outputs with placeholder message
with fig1_output:
    display(HTML(placeholder_html))
with fig2_output:
    display(HTML(placeholder_html))

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# SECTION 2: MAIN UPDATE FUNCTION
# ═════════════════════════════════════════════════════════════════════════════


def update_plots(*args) -> None:
    """Run the unified academic figure for the dashboard with enhanced feedback."""
    # Disable button and update progress
    run_button.disabled = True
    progress_label.value = "⏳ Running simulation... (15-30 seconds)"

    # Only show loading state in status label
    progress_label.value = "⏳ Computing Results... (15-30 seconds)"

    # Get parameter values
    params = {
        "phi_pi": phi_pi_widget.value,
        "phi_y": phi_y_widget.value,
        "rho_r": rho_r_widget.value,
        "kappa_p": kappa_p_widget.value,
        "phi_b": phi_b_widget.value,
        "real_wage_rigidity": real_wage_rigidity_widget.value,
        "UI_extension_length": ui_extension_widget.value,
        "tax_cut_length": tax_cut_widget.value,
    }

    try:
        # Run all experiments and get results
        results = hs.compute_fiscal_multipliers(**params)
        multipliers = results["multipliers"]
        irfs = results["irfs"]

        # Create figures with dashboard control over canvas
        import matplotlib.pyplot as plt

        # Figure 1: Fiscal Multipliers - guaranteed fit sizing
        fig1_output.clear_output(wait=True)
        with fig1_output:
            # Create figure that fits the container with matching background
            fig1, axes1 = plt.subplots(
                1, 3, figsize=(14, 3.5), sharey=True,
                facecolor=ARK_LIGHT  # Set figure background to match container
            )
            fig1.patch.set_alpha(1.0)  # Make background fully opaque
            plt.tight_layout(pad=1.5, rect=[0.02, 0.02, 0.98, 0.98])  # Adjust spacing and margins

            fig1 = hs.plot_multipliers_three_experiments(
                multipliers["transfers"],
                multipliers["transfers_fixed_nominal"],
                multipliers["transfers_fixed_real"],
                multipliers["UI_extend"],
                multipliers["UI_extend_fixed_nominal"],
                multipliers["UI_extend_fixed_real"],
                multipliers["tax_cut"],
                multipliers["tax_cut_fixed_nominal"],
                multipliers["tax_cut_fixed_real"],
                fig_and_axes=(fig1, axes1),
            )
            if fig1 is not None:
                display(fig1)
                plt.close(fig1)

        # Figure 2: Consumption IRFs - guaranteed fit sizing
        fig2_output.clear_output(wait=True)
        with fig2_output:
            # Create figure that fits the container with matching background
            fig2, axes2 = plt.subplots(
                1, 3, figsize=(14, 3.5), sharey=True,
                facecolor=ARK_LIGHT  # Set figure background to match container
            )
            fig2.patch.set_alpha(1.0)  # Make background fully opaque
            plt.tight_layout(pad=1.5, rect=[0.02, 0.02, 0.98, 0.98])  # Adjust spacing and margins

            fig2 = hs.plot_consumption_irfs_three_experiments(
                irfs["UI_extend"],
                irfs["UI_extend_fixed_nominal"],
                irfs["UI_extend_fixed_real"],
                irfs["transfer"],
                irfs["transfer_fixed_nominal"],
                irfs["transfer_fixed_real"],
                irfs["tau"],
                irfs["tau_fixed_nominal"],
                irfs["tau_fixed_real"],
                fig_and_axes=(fig2, axes2),
            )
            if fig2 is not None:
                display(fig2)
                plt.close(fig2)

        # Update summary statistics
        stimulus_mult_1yr = multipliers["transfers"][3]  # 1-year (4 quarters)
        ui_mult_1yr = multipliers["UI_extend"][3]
        tax_mult_1yr = multipliers["tax_cut"][3]

        summary_html = f"""
        <div style='display: flex; flex-direction: column; gap: 0.4em;
                    margin: 0; padding: 1em; background-color: {ARK_LIGHT}; border-radius: 8px;
                    color: {ARK_SLATE}; font-size: 0.9rem; line-height: 1.2;'>
            <div>Stimulus Check: <span style='color: {ARK_BLUE}; font-weight: 600; margin-left: 0.5em;'>{stimulus_mult_1yr:.2f}</span></div>
            <div>UI Extension: <span style='color: {ARK_BLUE}; font-weight: 600; margin-left: 0.5em;'>{ui_mult_1yr:.2f}</span></div>
            <div>Tax Cut: <span style='color: {ARK_BLUE}; font-weight: 600; margin-left: 0.5em;'>{tax_mult_1yr:.2f}</span></div>
        </div>
        """

        # Update the summary section (find and update the HTML widget)
        summary_section.children[1].value = summary_html

        progress_label.value = "✅ Simulation complete!"
        run_button.disabled = False

    except Exception as e:
        progress_label.value = f"❌ Error: {e!s}"
        run_button.disabled = False
        for output in [fig1_output, fig2_output]:
            with output:
                clear_output(wait=True)

In [ ]:
# Connect button to update function
run_button.on_click(update_plots)

In [ ]:
# ═════════════════════════════════════════════════════════════════════════════
# SECTION 3: CREATE DASHBOARD LAYOUT
# ═════════════════════════════════════════════════════════════════════════════

# SIMPLE SIDEBAR - focused on core functionality
options_panel = VBox(
    [
        create_heading("Model Parameters", 2, "lightblue"),
        create_heading("Monetary Policy", 3),
        phi_pi_widget,
        phi_y_widget,
        rho_r_widget,
        kappa_p_widget,
        create_heading("Fiscal & Structural", 3),
        phi_b_widget,
        real_wage_rigidity_widget,
        ui_extension_widget,
        tax_cut_widget,
        create_heading("Simulation", 3),
        status_panel,
    ],
    layout=Layout(
        border="none",
        padding="0.5em 0",  # Add vertical padding
        margin="1em 0",  # Add vertical margin
        width="100%",
        height="auto",  # Let it size to content
        min_height="600px",  # Ensure minimum height
        overflow_y="auto",  # Allow scrolling if needed
        overflow_x="hidden",  # No horizontal scroll
    ),
)

In [ ]:
# MAIN CONTENT - Two figure panels with fixed layout
fig1_panel = VBox(
    [
        create_heading("Fiscal Multipliers by Policy Type", 2, "lightblue"),
        fig1_output,
    ],
    layout=Layout(
        border="none",
        padding="0",
        margin="0 0 2em 0",  # Spacing between plots
        width="100%",
        height="400px",  # Even taller plot panels
        min_height="400px",  # Minimum height
        overflow="visible",  # Allow content to overflow
    ),
)

fig2_panel = VBox(
    [
        create_heading("Consumption Response Functions", 2, "lightblue"),
        fig2_output,
    ],
    layout=Layout(
        border="none",
        padding="0",
        margin="0",
        width="100%",
        height="400px",  # Even taller plot panels
        min_height="400px",  # Minimum height
        overflow="visible",  # Allow content to overflow
    ),
)

# Create introduction section with H1 title and larger body text
intro_section = VBox(
    [
        create_heading("HANK-SAM Fiscal Policy Analysis", 1),
        HTML(
            "<div style='margin: 0; padding: 0;'>"
            "<p class='ark-body' style='margin: 0 0 1em 0;'>"
            "This dashboard explores fiscal multipliers in a Heterogeneous Agent New Keynesian (HANK) model with Search and Matching frictions. "
            "The model features heterogeneous households facing ideosyncratic income risk, unemployment dynamics, and endogenous job creation, making it ideal for analyzing fiscal policy effectiveness.</p>"
            "<p class='ark-body' style='margin: 0 0 1em 0;'>"
            "Adjust the monetary and fiscal parameters below to explore how different policy regimes affect consumption multipliers. "
            "Compare results across three fiscal policies: stimulus checks, UI extensions, and tax cuts under standard Taylor rule, fixed nominal rate, and fixed real rate scenarios.</p>"
            "<p class='ark-body' style='margin: 0 0 1em 0; font-style: italic; opacity: 0.9;'>"
            "Key insight: UI extensions typically generate the highest multipliers due to targeting unemployed households with high marginal propensities to consume.</p>"
            "</div>"
        ),
    ],
    layout=Layout(
        width="100%",
        padding="0",
        margin="0 0 2em 0",
        min_height="250px",  # Minimum height
        height="auto",  # Let it grow as needed
        overflow="visible",  # Allow content to be fully visible
    ),
)

# Create summary statistics section (will be populated by simulation results)
summary_section = VBox(
    [
        create_heading("Key Multipliers (1-Year Horizon)", 2, "lightblue"),  # Same style as other headings
        HTML(
            "<div id='summary-stats' style='margin: 0.5em 0 0 0; padding: 1.5em; "
            f"background-color: {ARK_LIGHT}; border-radius: 8px; font-size: 1rem; text-align: center;' class='ark-label'>"
            "Run simulation to view key results...</div>"
        ),
    ],
    layout=Layout(
        width="100%",
        padding="0",
        margin="0 0 3em 0",  # More spacing after summary
        height="160px",  # Much more height for summary
        overflow="hidden",
    ),
)

# Create left panel with intro section above model parameters
left_panel = VBox(
    [intro_section, options_panel],
    layout=Layout(
        width="32%",  # Wider per brand guide
        height="auto",  # Let it size to content
        max_height="1100px",  # Stop after run simulation
        display="flex",
        flex_direction="column",
        padding="1em",
        background_color="#f5f7fa",
        border_right=f"4px solid {ARK_BLUE}",
    ),
)
right_panel = VBox(
    [summary_section, fig1_panel, fig2_panel],
    layout=Layout(
        width="68%",  # Adjusted to match left panel
        height="100%",
        padding="0.6em",
        background_color="white",
        overflow="hidden",  # NO scrollbars allowed
        display="flex",  # Explicit flexbox
        flex_direction="column",  # Stack children vertically
        gap="0.2em",  # Small gap between elements
    ),
)

# Split horizontally: Options left (30%) -> Figures right (70%)
main_content = HBox(
    [left_panel, right_panel],
    layout=Layout(
        width="100%",
        height="1400px",  # Extra tall dashboard
        overflow="hidden",  # Prevent outer scrollbars
        margin="0",
        padding="0",
    ),
)

# Complete dashboard
dashboard = VBox(
    [main_content],
    layout=Layout(
        width="100%",
        height="1400px",  # Extra tall dashboard
        overflow="hidden",  # Master overflow control - NO SCROLLBARS
        margin="0",
        padding="0",
    ),
)

In [ ]:
# Initialize with welcome message
with fig1_output:
    pass

In [ ]:
# Display dashboard
dashboard